In [1]:
from langchain_groq import ChatGroq

In [2]:
llm=ChatGroq(
    temperature=0,
    model_name="mixtral-8x7b-32768",
    groq_api_key='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'
)
response=llm.invoke("who is the current pm of india")
print(response.content)

As of my knowledge up to October 2021, the current Prime Minister of India is Narendra Modi. He has been in office since May 2014. However, I recommend checking the most recent and reliable sources to confirm, as political situations can change rapidly.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://amazon.jobs/en/jobs/2816887/software-dev-engineer-i-amazon-university-talent-acquisition")
page_data=loader.load().pop().page_content
print(page_data)


USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Dev Engineer I, Amazon University Talent Acquisition - Job ID: 2816887 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesYour job applicationResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×Software Dev Engineer I, Amazon University Talent AcquisitionJob ID: 2816887 | ADCI - KarnatakaApply nowDESCRIPTIONAt Amazon, we hire the best minds in technology to innovate and build on behalf of our customers. The focus we have on our customers is why we are one of the world’s most beloved brands – customer obsession is part of our company DNA. Our Software Development Engineers (SDEs) use cutting-edge technology to solve complex problems and get to see the impact of their work first-hand. The challenges SDEs solve for at Amazon are big and influence millions of customers, sellers, and products around the world. We are looking for individuals who are passionate about creating new products, features, and

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    ### INSTRUCTION
    The scrapped text is from the career's page of a website.
    Your job is to extract the job posting and return them in JSON format containing 
    following keys:'role','experience','skills' and 'description'.
    Only return the Valid JSON
    ### VALID JSON (NO PREAMBLE)
    """
    
)
chain_extract=prompt_extract |  llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)


{
  "role": "Software Dev Engineer I",
  "experience": "Not specified",
  "skills": [
    "Collaboration with cross-disciplinary teams",
    "Designing and building technologies in a large distributed computing environment",
    "Creating solutions for distributed systems with exposure to innovative technologies at scale and speed",
    "Building distributed storage, index, and query systems that are scalable, fault-tolerant, low cost, and easy to manage/use",
    "Designing and coding the right solutions starting with broadly defined problems",
    "Working in an agile environment to deliver high-quality software"
  ],
  "description": "At Amazon, we hire the best minds in technology to innovate and build on behalf of our customers. The focus we have on our customers is why we are one of the world’s most beloved brands – customer obsession is part of our company DNA. Our Software Development Engineers (SDEs) use cutting-edge technology to solve complex problems and get to see the impa

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res
# type(json_res)

{'role': 'Software Dev Engineer I',
 'experience': 'Not specified',
 'skills': ['Collaboration with cross-disciplinary teams',
  'Designing and building technologies in a large distributed computing environment',
  'Creating solutions for distributed systems with exposure to innovative technologies at scale and speed',
  'Building distributed storage, index, and query systems that are scalable, fault-tolerant, low cost, and easy to manage/use',
  'Designing and coding the right solutions starting with broadly defined problems',
  'Working in an agile environment to deliver high-quality software'],
 'description': 'At Amazon, we hire the best minds in technology to innovate and build on behalf of our customers. The focus we have on our customers is why we are one of the world’s most beloved brands – customer obsession is part of our company DNA. Our Software Development Engineers (SDEs) use cutting-edge technology to solve complex problems and get to see the impact of their work first-h

In [7]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import chromadb
import uuid

client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])
        
    

In [18]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [17]:
job=json_res
job['skills']

['Collaboration with cross-disciplinary teams',
 'Designing and building technologies in a large distributed computing environment',
 'Creating solutions for distributed systems with exposure to innovative technologies at scale and speed',
 'Building distributed storage, index, and query systems that are scalable, fault-tolerant, low cost, and easy to manage/use',
 'Designing and coding the right solutions starting with broadly defined problems',
 'Working in an agile environment to deliver high-quality software']

In [23]:
prompt_email=PromptTemplate.from_template(
    """
    ###JOB DESCRIPTION
    {job_description}

    ### INSTRUCTION
    You are Yash Agarwal,a fresher software Engineer currently in 4th year of btech at jadavpur University
    I have a past experience of interning at Bank of new York  where i worked in backend team 
    Your job is to write a cold email to the HR regarding the job mentioned above fullfilling their needs
    Also add the most relevant ones from the following link to showcase skills {link_list}
    Do not provide a preamble 
    ###Email (no preamble)
    """
)

chain_email=prompt_email | llm
res=chain_email.invoke({"job_description":str(job),"link_list":links})
print(res.content)

Subject: Application for Software Dev Engineer I Position

Dear Hiring Manager,

I am writing to express my interest in the Software Dev Engineer I position at Amazon, as advertised. As a fourth-year B.Tech student at Jadavpur University with a past internship experience at Bank of New York, where I worked in the backend team, I am confident that I possess the skills and experience necessary to excel in this role.

The job description mentions the need for skills such as collaboration with cross-disciplinary teams, designing and building technologies in a large distributed computing environment, and building distributed storage, index, and query systems. I have developed these skills through my past experience and academic projects.

I have a portfolio of work that demonstrates my proficiency in DevOps, which I believe is highly relevant to this role. You can view my portfolio at <https://example.com/devops-portfolio>. Additionally, I have experience working with backend technologies s